In [1]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')

In [5]:
import pandas as pd
import numpy as np

In [28]:
headers = ["business_id", "stars", "useful", "funny", "cool", "text", "date"]
dtype = {"business_id" : str, 
         "stars" : np.single, 
         "useful" : np.single, 
         "funny" : np.single, 
         "cool" : np.single, 
         "text" : str, 
         "date" : str}
parse_dates = ['date']
date_parser = lambda x: pd.datetime.strptime(x, "%Y %m %d %H:%M:%S")
df = pd.read_csv('data/reviews.csv', header=0, names=headers, dtype=dtype, parse_dates=parse_dates, date_parser=date_parser)

NameError: name 'O' is not defined